In [35]:


import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import hashlib
import numpy as np

# recommendations
import surprise
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.prediction_algorithms import BaselineOnly, KNNBaseline, Prediction
from surprise import SVD

# metrics
import ml_metrics as metrics

# eda
#from data_utils import describe, transformers

# visualization
from matplotlib import pyplot as plt



### Explore

In [36]:
!ls -lah data/

total 453776
drwxr-xr-x  11 sofiacardita  staff   374B Aug 26 10:04 .
drwxr-xr-x   9 sofiacardita  staff   306B Aug 26 10:34 ..
-rw-r--r--@  1 sofiacardita  staff   6.0K Aug 26 10:04 .DS_Store
-rw-r--r--   1 sofiacardita  staff   6.6K Aug 26 09:55 example_output.csv
-rw-r--r--   1 sofiacardita  staff    77M Jul 28 12:30 song_tag.csv
-rw-r--r--   1 sofiacardita  staff    19M Aug 26 09:55 song_tag.zip
-rw-r--r--   1 sofiacardita  staff   9.5M Aug 26 09:55 songs.txt
-rw-r--r--   1 sofiacardita  staff    12M Aug 26 09:55 tags.csv
-rw-r--r--   1 sofiacardita  staff   400K Aug 26 09:55 test_users.csv
-rw-r--r--   1 sofiacardita  staff    86M Mar 11  2012 train_play_counts.txt
-rw-r--r--   1 sofiacardita  staff    18M Aug 26 09:55 train_play_counts.zip


In [3]:
!head data/train_play_counts.txt

fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOBONKR12A58A7A7E0	1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOEGIYH12A6D4FC0E3	1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOFLJQZ12A6D4FADA6	1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOHTKMO12AB01843B0	1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SODQZCY12A6D4F9D11	1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOXLOQG12AF72A2D55	1
d7083f5e1d50c264277d624340edaaf3dc16095b	SOUVUHC12A67020E3B	1
d7083f5e1d50c264277d624340edaaf3dc16095b	SOUQERE12A58A75633	1
d7083f5e1d50c264277d624340edaaf3dc16095b	SOIPJAX12A8C141A2D	1
d7083f5e1d50c264277d624340edaaf3dc16095b	SOEFCDJ12AB0185FA0	2


In [4]:
!head data/songs.txt

SOAAADD12AB018A9DD 1
SOAAADE12A6D4F80CC 2
SOAAADF12A8C13DF62 3
SOAAADZ12A8C1334FB 4
SOAAAFI12A6D4F9C66 5
SOAAAGK12AB0189572 6
SOAAAGN12AB017D672 7
SOAAAGO12A67AE0A0E 8
SOAAAGP12A6D4F7D1C 9
SOAAAGQ12A8C1420C8 10


In [42]:
def make_ratings(path):

    users = make_users()
    items = make_items()
dd
    users_ = read_array_from_csv(path, 'object', 0, delimdditer="\t")
    items_ = read_array_from_csv(path, 'object', 1)

    rows = make_rows(users, users_)
    cols = make_cols(items, items_)

    nrows = users.shape[0]
    ncols = items.shape[0]

    shape = (nrows, ncols)

    data = np.ones(rows.size)

    return coo_matrix((data, (rows, cols)), shape=shape)


def make_users():
    path = os.path.join('data', 'train_play_counts.txt')
    users = read_array_from_csv(path, 'object', 0, delimiter="\t")
    return users[users.argsort()]


def make_items():
    path = os.path.join('data', 'songs.txt')
    items = read_array_from_csv(path, 'object', 0, delimiter=" ")
    return items[items.argsort()]


def read_array_from_csv(path, dtype, column, delimiter=" "):
    return np.genfromtxt(path, dtype=dtype, skip_header=False, usecols=[column],
                         delimiter=delimiter)

def make_rows(users, users_):
    rows = [np.argwhere(users == u)[0, 0] for u in users_]
    return np.array(rows)


def make_cols(items, items_):
    cols = [np.argwhere(items == i)[0, 0] for i in items_]
    return np.array(cols)

In [ ]:
R = make_ratings("data/train_play_counts.txt")